<a href="https://colab.research.google.com/github/tevfikaytekin/data_science/blob/master/recommender_systems/matrix_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix Factorization
(by Tevfik Aytekin)

Matrix factorizarion is one of the state-of-the-art techniques used in recommender systems. Below you can find several different implementations.

Over the years many variations of matrix factorization have been proposed. The following formulation is one of the simplest but which works well as we will see. It can be extended it various ways, see for example [Advances in Collaborative Filtering](https://link.springer.com/chapter/10.1007/978-1-4899-7637-6_3).

Cost Function:
$$
J(\Theta) =  \sum_{u,i \in K} (r_{ui} - q^T_ip_u)^2 + \lambda(||q_i||^2+||p_u||^2)
$$

where

- $r_{ui}$ is the rating of user $u$ for item $i$.
- $K$ is the set of $(u,i)$ pairs for which $r_{ui}$ is known.
- $q_i$, $p_u$ are latent factor vectors for items and users, respectively. 
- $\lambda$ is the regularization parameter.

And the optimization objective:

$$
\min_{p*,q*} \sum_{u,i \in K} (r_{ui} - q^T_ip_u)^2 + \lambda(||q_i||^2+||p_u||^2)
$$

Typically the optimization done with gradient descent. To apply it we need to first find the partial derivative of the cost function with respect to latent variables which we will denote as $q_{fi}$ and $p_{fu}$. We can find the partial derivative as: 

$$
\frac{\partial J(\Theta)}{\partial p_{ku}}=-\sum_{i \in I_u}2(r_{ui} - q^T_ip_u)q_{ki} + 2\lambda p_{ku}
$$

For **stochastic gradient descent** the update rule for the the $p_u$ vector for a single training example is:

$$
p_u = p_u + \alpha ((r_{ui} - q^T_ip_u)q_{i} - \lambda p_{u})
$$

similarly for $q_i$ vector we have:

$$
q_i = q_i + \alpha ((r_{ui} - q^T_ip_u)p_{u} - \lambda q_{u})
$$

For **batch gradient descent** the update rule for the the $p_u$ vector for all preferences where user $u$ appears:

$$
p_u = p_u + \alpha (\sum_{i \in I_u}(r_{ui} - q^T_ip_u)q_{i} - \lambda p_{u})
$$

similarly for $q_i$ vector we have:

$$
q_i = q_i + \alpha (\sum_{u \in U_i} (r_{ui} - q^T_ip_u)p_{u} - \lambda q_{u})
$$

In the above equations $I_u$ is the set of items rated by user $u$ and $U_i$ is the set of users who rated item $i$.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix
import copy

# Movielens dataset

We will use the smallest Movielens 100k Dataset which includes 100k preferences. A preference is a triple (user, item, rating). You can download this data set from
[https://grouplens.org/datasets/movielens/](https://grouplens.org/datasets/movielens/)

Note the sparsity of the dataset which shows that most of the user/item matrix is empty. This is a typical property of the datasets in this domain.

In [3]:
prefs = pd.read_csv("/home/tevfik/Documents/datasets/ml-latest-small/ratings.csv", sep=",")
#prefs = pd.read_csv("ratings.csv", sep=",")
#prefs = pd.read_csv("drive/MyDrive/PycharmProjects/datasets/ml-latest-small/ratings.csv", sep=",")

prefs.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
n_users = prefs.iloc[:,0].unique().size
n_items = prefs.iloc[:,1].unique().size
n_prefs = prefs.iloc[:,1].size
users = prefs.iloc[:,0].unique()
items = prefs.iloc[:,1].unique()

print("Number of users:",n_users)
print("Number of items:",n_items)
print("Number of preferences:",n_prefs)
print("Sparsity:",n_prefs/(n_users*n_items))

Number of users: 610
Number of items: 9724
Number of preferences: 100836
Sparsity: 0.016999683055613623


### Error Function

Error is calculated by predicting the rating of a user and an item in the test set using the factor representations of users and items.

In [5]:
def calc_error(X, u_factors, i_factors):
    error = 0
    for i in range(X.shape[0]):
        u_idx = X.iloc[i,0]
        i_idx = X.iloc[i,1]
        error += np.abs(X.iloc[i,2] - np.dot(u_factors[u_idx].T, i_factors[i_idx]))
    return error/X.shape[0]
    

### Random Predictor Error

**Exercise**: What is the expected error of a random predictor given that the actual ratings are uniformly distributed between 1 and 5?

Below is a function for calculating this error experimentally. 

In [6]:
def random_predictor_error(X):
    error = 0
    for i in range(X.shape[0]):
        u_idx = X.iloc[i,0]
        i_idx = X.iloc[i,1]
        error += np.abs(X.iloc[i,2] - np.random.randint(1,6))
    return error/X.shape[0]

In [9]:
# initialize factor matrices
n_factors = 5
item_factors = {}
user_factors = {}
for r in range(n_prefs):
    user_factors[prefs.iloc[r,0]] = np.random.rand(n_factors,1) - 0.5
    item_factors[prefs.iloc[r,1]] = np.random.rand(n_factors,1) - 0.5

In [10]:
user_factors[1]

array([[-0.11979976],
       [-0.09314695],
       [-0.42510215],
       [ 0.09456017],
       [-0.32070388]])

In [7]:
print("Random predictor error: ", random_predictor_error(prefs))

Random predictor error:  1.4802749018207784


### Stochastic Gradient Descent

Following is the stochastic gradient algorithm which is popularized by [Simon Funk](https://sifter.org/simon/journal/20061211.html)

In [19]:
n_factors = 5
item_factors = {}
user_factors = {}
for r in range(n_prefs):
    user_factors[prefs.iloc[r,0]] = np.random.rand(n_factors,1) - 0.5
    item_factors[prefs.iloc[r,1]] = np.random.rand(n_factors,1) - 0.5
    
X_train, X_test = train_test_split(prefs, test_size=0.1)

# Stochastic Gradient descent
alpha = 0.03
my_lambda = 0.1
n_iters = 5
    
print("Initial error: ", calc_error(X_train, user_factors, item_factors))

for t in range(n_iters):
    #q = 10
    #ux = prefs.iloc[q,0]; ix = prefs.iloc[q,1]
    #print("user factor: ",user_factors[ux],"item factor: ", item_factors[ix])
    #print("actual rating: ", prefs.iloc[q,2], "predicted rating: ", np.dot(user_factors[ux].T,item_factors[ix]))
    X_train = shuffle(X_train)
    for r in range(X_train.shape[0]):
        u = X_train.iloc[r,0]
        i = X_train.iloc[r,1]
        error = X_train.iloc[r,2] - np.dot(user_factors[u].T, item_factors[i])[0,0]
        user_factors[u] = user_factors[u] + alpha*(error*item_factors[i] - my_lambda*user_factors[u])
        item_factors[i] = item_factors[i] + alpha*(error*user_factors[u] - my_lambda*item_factors[i])  
       
          
    print("Iteration ", t)
    print("Train error: ", calc_error(X_train, user_factors, item_factors))
    print("Test error: ", calc_error(X_test, user_factors, item_factors))
    

Initial error:  [[3.50119857]]
Iteration  0
Train error:  [[1.04724026]]
Test error:  [[1.12198071]]
Iteration  1
Train error:  [[0.80947548]]
Test error:  [[0.91300559]]
Iteration  2
Train error:  [[0.73427169]]
Test error:  [[0.85732055]]
Iteration  3
Train error:  [[0.69932492]]
Test error:  [[0.83645428]]
Iteration  4
Train error:  [[0.67963575]]
Test error:  [[0.82241197]]


### How to make a prediction?
Once the user and item factors are learned you can make a prediction for any user and item pair.

In [37]:
item_factors[10]

array([[ 1.0223775 ],
       [-0.3111142 ],
       [-0.80764908],
       [ 0.12634261],
       [ 1.07515975]])

In [38]:
user_factors[100]

array([[ 1.56532791],
       [-0.12872923],
       [-1.21287299],
       [ 0.2411651 ],
       [ 1.03796345]])

In [39]:
np.dot(user_factors[10].T, item_factors[50])

array([[3.86508616]])

### Batch Gradient Descent
If you run the code below you will see that both training and test errors decrease very slowly. Eventually there will be convergence but compared to stochastic version it will be very slow. It is a good example to show the speed advantage of stochastic gradient descent.

In [40]:
from scipy.sparse import csr_matrix
import copy
#initialize factor matrices
item_factors = {}
user_factors = {}
for i in range(n_prefs):
    user_factors[prefs.iloc[i,0]] = np.random.rand(n_factors,1) - 0.5
    item_factors[prefs.iloc[i,1]] = np.random.rand(n_factors,1) - 0.5
    
X_train, X_test = train_test_split(prefs, test_size=0.1)

train_users = X_train.iloc[:,0].unique()
train_items = X_train.iloc[:,1].unique()
R = csr_matrix((X_train.iloc[:,2], (X_train.iloc[:,0],X_train.iloc[:,1])))

# Batch Gradient descent
alpha = 0.1
my_lambda = 0.1
n_iters = 100

for t in range(n_iters):
    for u in train_users:
        I_u = X_train[X_train.userId==u].iloc[:,1]
        sum_total = 0
        for i in I_u:
            sum_total += (R[u,i] - np.dot(item_factors[i].T, user_factors[u])[0,0])*item_factors[i]
        sum_total = sum_total / I_u.size
        user_factors[u] = user_factors[u] + alpha*(sum_total - my_lambda*user_factors[u])
    for i in train_items:
        U_i = X_train[X_train.movieId==i].iloc[:,0]
        sum_total = 0
        for u in U_i:
            sum_total += (R[u,i] - np.dot(item_factors[i].T, user_factors[u])[0,0])*user_factors[u]
        sum_total = sum_total / U_i.size
        item_factors[i] = item_factors[i] + alpha*(sum_total - my_lambda*item_factors[i])
        
    print("Iteration ", t)
    print("Train error: ", calc_error(X_train,user_factors,item_factors))
    print("Test error: ", calc_error(X_test,user_factors,item_factors))

Iteration  0
Train error:  [[3.48679096]]
Test error:  [[3.50851927]]
Iteration  1
Train error:  [[3.47311008]]
Test error:  [[3.50814783]]
Iteration  2
Train error:  [[3.45820314]]
Test error:  [[3.50708082]]
Iteration  3


KeyboardInterrupt: 

### Question

Suppose that user A appears in 200 rows in the user-item preferences dataset. In a single epoch how many updates will there be to the latent vector $p_u$ in stochastic GD vs. Batch GD?

### Alternating Least Squares (ALS)
Since the datasets in this domain are really large, people always try to find ways to speed up the optimization processes. One popular algorithm is called ALS. Here the basic idea is that although the cost function is not convex, when either user factors or items factors are fixed then it becomes a convex function which can be directly solved. The algorithm alternates between updating user and item factors. ([Large-scale parallel collaborative filtering for the netflix prize](https://link.springer.com/chapter/10.1007/978-3-540-68880-8_32)). 

$$
\frac{\partial J(\Theta)}{\partial p_{ku}}=-\sum_{i \in I_u}2(r_{ui} - q^T_ip_u)q_{ki} + 2\lambda p_{ku} = 0
$$

$$
\sum_{i \in I_u} q_{ki}q^T_ip_u + \lambda p_{ku} = \sum_{i \in I_u}q_{ki}r_{ui} 
$$

$$
\sum_{i \in I_u} q_{i}q^T_ip_u + \lambda p_{u} = \sum_{i \in I_u}q_{i}r_{ui} 
$$

$$
(Q_{I_u}Q_{I_u}^T + \lambda I) p_{u} = Q_{I_u}R^T(u,I_u)
$$

$$
p_{u} = (Q_{I_u}Q_{I_u}^T + \lambda I)^{-1}Q_{I_u}R^T(u,I_u)
$$

where $I$ is the $f × f$ identity matrix. $Q_{I_u}$ denotes the sub-matrix of $Q$ where columns $j \in I_u$ are selected, and $R^T(u,I_u)$ is the row vector where columns $j \in I_u$ of the $u$-th row of $R$ are selected.

Similarly for $q_i$ we have:

$$
q_{i} = (P_{U_i}P_{U_i}^T + \lambda I)^{-1}P_{U_i}R(U_i,i)
$$

where $I$ is the $f × f$ identity matrix. $P_{U_i}$ denotes the sub-matrix of $P$ where columns $j \in U_i$ are selected, and $R(U_i,i)$ is the column vector where columns $j \in U_i$ of the $i$-th column of $R$ are selected.



In [41]:
#initialize factor matrices
n_factors = 5

Q = pd.DataFrame(np.random.rand(n_factors,n_items)-0.5, columns=items)
P = pd.DataFrame(np.random.rand(n_factors,n_users)-0.5, columns=users)

X_train, X_test = train_test_split(prefs, test_size=0.1)

train_users = X_train.iloc[:,0].unique()
train_items = X_train.iloc[:,1].unique()
R = csr_matrix((X_train.iloc[:,2], (X_train.iloc[:,0],X_train.iloc[:,1])))

alpha = 0.030
my_lambda = 0.1
n_iters = 100

for t in range(n_iters):
    for u in train_users:
        I_u = X_train[X_train.userId==u].iloc[:,1]
        A = np.dot(Q[I_u],Q[I_u].T)+my_lambda*np.identity(n_factors)
        V = np.dot(Q[I_u],R[u,I_u].todense().T)
        P[u] = np.dot(np.linalg.inv(A),V)
    for i in train_items:
        U_i = X_train[X_train.movieId==i].iloc[:,0]
        A = np.dot(P[U_i],P[U_i].T)+my_lambda*np.identity(n_factors)
        V = np.dot(P[U_i],R[U_i,i].todense())     
        Q[i] = np.dot(np.linalg.inv(A),V)
        
    print("Iteration ", t)
    print("Train error: ", calc_error(X_train,P,Q))
    print("Test error: ", calc_error(X_test,P,Q))

Iteration  0
Train error:  2.007204001449057
Test error:  3.0777682475972528
Iteration  1
Train error:  0.8191487562109738
Test error:  1.2703678764134934


KeyboardInterrupt: 

In [35]:
## The following has not yet finished

class MF:
    """
    prefs: matrix of prefences, column0=userid, column1=itemid, column2=pref, column3=timestamp 
    """
    def __init__(self, prefs, alpha=0.03, mylambda=0.1, n_factors = 10, n_iters = 50):
        self.alpha = alpha
        self.mylambda = mylambda
        self.n_iters = n_iters
        self.item_factors = {}
        self.user_factors = {}
        self.prefs = prefs
        # prefs is a matrix containing u, i, r values in each row. This is useful to shuffle and pass over
        # the data multiple times in an efficient way in the fit() method.
        for r in range(self.prefs.shape[0]):
            self.user_factors[self.prefs.iloc[r,0]] = np.random.rand(n_factors,1) - 0.5
            self.item_factors[self.prefs.iloc[r,1]] = np.random.rand(n_factors,1) - 0.5
        print("Finished initialization")
        
     
    def calc_error(self, X):
        error = 0
        for i in range(X.shape[0]):
            u_idx = X.iloc[i,0]
            i_idx = X.iloc[i,1]
            error += np.abs(X.iloc[i,2] - np.dot(self.user_factors[u_idx].T, self.item_factors[i_idx]))
        return error/self.prefs.shape[0]
        
    def fit(self, verbose=False, method="SGD"):
        if (method == "Random"):
            error = 0
            for i in range(prefs.shape[0]):
                u_idx = prefs.iloc[i,0]
                i_idx = prefs.iloc[i,1]
                error += np.abs(prefs.iloc[i,2] - np.random.randint(1,6))
            return error/prefs.shape[0]
            
        elif (method == "SGD"):
            if (verbose): 
                print("Initial error: ", self.calc_error(prefs))                      
            for t in range(self.n_iters):
                self.prefs = shuffle(self.prefs)
                for r in range(self.prefs.shape[0]):
                    u = self.prefs.iloc[r,0]
                    i = self.prefs.iloc[r,1]
                    error = self.prefs.iloc[r,2] - np.dot(self.user_factors[u].T, self.item_factors[i])[0,0]
                    self.user_factors[u] = self.user_factors[u] + self.alpha*(error*self.item_factors[i] - self.mylambda*self.user_factors[u])
                    self.item_factors[i] = self.item_factors[i] + self.alpha*(error*self.user_factors[u] - self.mylambda*self.item_factors[i])  
            
                if (verbose): 
                    print("Iteration: ", t)
                if (verbose): 
                    print("Train error: ", self.calc_error(self.prefs))                      


In [34]:
mf = MF(prefs)
mf.fit(verbose=True, method="SGD")

Finished initialization
Random Predictor Error: [[3.50133057]]
Initial error:  None
Iteration:  0
Random Predictor Error: [[0.97004331]]
Train error:  None
Iteration:  1
Random Predictor Error: [[0.77222596]]
Train error:  None
Iteration:  2
Random Predictor Error: [[0.71212855]]
Train error:  None
Iteration:  3
Random Predictor Error: [[0.67955195]]
Train error:  None
Iteration:  4
Random Predictor Error: [[0.65899879]]
Train error:  None
Iteration:  5
Random Predictor Error: [[0.63166851]]
Train error:  None
Iteration:  6
Random Predictor Error: [[0.62009774]]
Train error:  None
Iteration:  7
Random Predictor Error: [[0.60613902]]
Train error:  None
Iteration:  8
Random Predictor Error: [[0.59523205]]
Train error:  None
Iteration:  9


KeyboardInterrupt: 

In [ ]:
X_train, X_test = train_test_split(prefs, test_size=0.1)
mf = MF(X_train, n_iters=3)
mf.fit(verbose=True, method="SGD")
print("Test error: ", mf.calc_error(X_test)) 